In [ ]:
import urllib.parse
import pandas as pd
import numpy as np
import os, json, requests

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_df = {
    'district': ["Ahmednagar","Akola","Amravati","Aurangabad","Beed","Bhandara","Buldhana","Chandrapur","Dhule","Gadchiroli","Gondia",
            "Hingoli","Jalgaon","Jalna","Kolhapur","Latur","Mumbai City","Mumbai Suburban","Nagpur","Nanded","Nandurbar","Nashik",
            "Osmanabad","Palghar","Parbhani","Pune","Raigad","Ratnagiri","Sangli","Satara","Sindhudurg","Solapur","Thane","Wardha",
            "Washim","Yavatmal"],

    'lat_long': [(19.093412,74.746855),(20.710576,77.00373),(20.937346,77.760249),(19.885543,75.333441),(18.987824,75.763809),
            (21.169245,79.657152),(20.532223,76.181689),(19.951685,79.295823),(20.90441,74.781243),(20.184794,80.007887),
            (21.448734,80.1972),(19.713154,77.153409),(21.009559,75.570044),(19.848844,75.901627),(16.694394,74.22406),
            (18.401122,76.576955),(18.987074,72.830007),(19.054135,72.832678),(21.148204,79.096814),(19.159314,77.313188),
            (21.36675,74.244736),(20.006006,73.795878),(18.181663,76.041686),(19.7867,72.9933),(19.268358,76.777025),
            (18.525994,73.862602),(18.646539,72.875994),(16.990597,73.297537),(16.860757,74.57878),(17.690393,74.010744),
            (16.107985,73.714977),(17.672099,75.907906),(19.205931,72.971198),(20.735221,78.604456),(20.108935,77.142117),
            (20.3876,78.131472)]
}

maha_districts = pd.DataFrame(data_df)

In [ ]:
maha_districts

,district,lat_long
0,Ahmednagar,"(19.093412, 74.746855)"
1,Akola,"(20.710576, 77.00373)"
2,Amravati,"(20.937346, 77.760249)"
3,Aurangabad,"(19.885543, 75.333441)"
4,Beed,"(18.987824, 75.763809)"
5,Bhandara,"(21.169245, 79.657152)"
6,Buldhana,"(20.532223, 76.181689)"
7,Chandrapur,"(19.951685, 79.295823)"
8,Dhule,"(20.90441, 74.781243)"
9,Gadchiroli,"(20.184794, 80.007887)"


No charts were generated by quickchart


In [ ]:
maha_districts.dtypes

district    object
lat_long    object
dtype: object

In [ ]:
state_climate_data = pd.DataFrame()
district_climate_data = pd.DataFrame()

output = r""

# base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters='PS','TS','T2M','QV2M','RH2M','WD2M','WS2M','T2MDEW','T2MWET','GWETTOP','GWETPROF','GWETROOT','WS2M_MAX','WS2M_MIN','WS2M_RANGE','PRECTOTCORR','PRECTOTCORR_SUM'&community=RE&longitude={longitude}&latitude={latitude}&start=2001&end=2021&format=JSON"

base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=GWETTOP,GWETROOT,GWETPROF,PS,WS2M,WS2M_MAX,WS2M_MIN,WS2M_RANGE,WD2M,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,QV2M,RH2M,PRECTOTCORR,PRECTOTCORR_SUM&community=AG&longitude={longitude}&latitude={latitude}&format=JSON&start=2001&end=2021"

for district, co_ordinates  in maha_districts.values:
    print(district, co_ordinates)

    latitude, longitude = co_ordinates[0], co_ordinates[1]

    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    content = json.loads(response.content.decode('utf-8'))

    filename = response.headers['content-disposition'].split('filename=')[1]

    district_climate_data = pd.DataFrame(content['properties']['parameter']).reset_index().rename(columns={'index': 'year_month'})

    district_climate_data['district'] = district

    state_climate_data = pd.concat([state_climate_data, district_climate_data], ignore_index=True, axis=0)

    # filepath = os.path.join(output, filename)
    # with open(filepath, 'w') as file_object:
    #     json.dump(content, file_object)

Ahmednagar (19.093412, 74.746855)
Akola (20.710576, 77.00373)
Amravati (20.937346, 77.760249)
Aurangabad (19.885543, 75.333441)
Beed (18.987824, 75.763809)
Bhandara (21.169245, 79.657152)
Buldhana (20.532223, 76.181689)
Chandrapur (19.951685, 79.295823)
Dhule (20.90441, 74.781243)
Gadchiroli (20.184794, 80.007887)
Gondia (21.448734, 80.1972)
Hingoli (19.713154, 77.153409)
Jalgaon (21.009559, 75.570044)
Jalna (19.848844, 75.901627)
Kolhapur (16.694394, 74.22406)
Latur (18.401122, 76.576955)
Mumbai City (18.987074, 72.830007)
Mumbai Suburban (19.054135, 72.832678)
Nagpur (21.148204, 79.096814)
Nanded (19.159314, 77.313188)
Nandurbar (21.36675, 74.244736)
Nashik (20.006006, 73.795878)
Osmanabad (18.181663, 76.041686)
Palghar (19.7867, 72.9933)
Parbhani (19.268358, 76.777025)
Pune (18.525994, 73.862602)
Raigad (18.646539, 72.875994)
Ratnagiri (16.990597, 73.297537)
Sangli (16.860757, 74.57878)
Satara (17.690393, 74.010744)
Sindhudurg (16.107985, 73.714977)
Solapur (17.672099, 75.907906)
Th

In [ ]:
state_climate_data

,year_month,PS,QV2M,RH2M,WD2M,WS2M,WD10M,WS10M,GWETTOP,GWETPROF,GWETROOT,WS2M_MAX,WS2M_MIN,WS10M_MAX,WS10M_MIN,WS2M_RANGE,PRECTOTCORR,WS10M_RANGE,PRECTOTCORR_SUM,district
0,200101,94.46,5.68,34.00,81.75,1.52,76.00,2.37,0.30,0.60,0.59,3.75,0.21,4.84,0.30,3.54,0.01,4.55,0.00,Ahmednagar
1,200102,94.37,3.72,20.81,327.81,1.87,331.56,2.89,0.16,0.60,0.59,4.71,0.08,6.53,0.09,4.63,0.00,6.45,0.00,Ahmednagar
2,200103,94.26,6.47,31.31,284.75,2.63,286.69,3.66,0.14,0.60,0.59,6.00,0.16,7.84,0.28,5.84,0.03,7.55,0.00,Ahmednagar
3,200104,94.14,7.45,26.31,345.19,2.10,343.81,2.90,0.18,0.60,0.59,5.02,0.02,6.83,0.02,5.00,0.16,6.81,0.00,Ahmednagar
4,200105,93.89,11.47,42.81,272.44,4.43,272.44,5.75,0.17,0.60,0.59,7.20,0.48,9.60,0.54,6.72,0.18,9.06,0.00,Ahmednagar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9823,202109,96.76,19.47,87.69,269.12,2.00,268.12,3.01,0.92,0.91,0.91,5.93,0.05,8.53,0.08,5.88,11.43,8.45,342.76,Yavatmal
9824,202110,97.29,15.75,79.44,37.75,1.36,37.50,2.16,0.90,0.91,0.91,4.78,0.07,6.80,0.07,4.71,3.77,6.73,116.95,Yavatmal
9825,202111,97.54,13.61,81.56,69.50,1.66,69.00,2.65,0.80,0.79,0.78,3.88,0.20,5.39,0.27,3.68,0.17,5.12,5.20,Yavatmal
9826,202112,97.88,9.70,70.38,70.56,1.45,69.88,2.30,0.70,0.70,0.69,4.11,0.10,5.66,0.12,4.01,0.35,5.54,10.77,Yavatmal


In [ ]:
month_dict = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December',
    13: 'YEAR AVG'
}

state_climate_data['year'] = state_climate_data['year_month'].map(lambda x: int(str(x)[:4]))
state_climate_data['month'] = state_climate_data['year_month'].map(lambda x: month_dict[int(str(x)[-2:])])

In [ ]:
state_climate_data

,year_month,PS,QV2M,RH2M,WD2M,WS2M,WD10M,WS10M,GWETTOP,GWETPROF,...,WS2M_MIN,WS10M_MAX,WS10M_MIN,WS2M_RANGE,PRECTOTCORR,WS10M_RANGE,PRECTOTCORR_SUM,district,year,month
0,200101,94.46,5.68,34.00,81.75,1.52,76.00,2.37,0.30,0.60,...,0.21,4.84,0.30,3.54,0.01,4.55,0.00,Ahmednagar,2001,January
1,200102,94.37,3.72,20.81,327.81,1.87,331.56,2.89,0.16,0.60,...,0.08,6.53,0.09,4.63,0.00,6.45,0.00,Ahmednagar,2001,February
2,200103,94.26,6.47,31.31,284.75,2.63,286.69,3.66,0.14,0.60,...,0.16,7.84,0.28,5.84,0.03,7.55,0.00,Ahmednagar,2001,March
3,200104,94.14,7.45,26.31,345.19,2.10,343.81,2.90,0.18,0.60,...,0.02,6.83,0.02,5.00,0.16,6.81,0.00,Ahmednagar,2001,April
4,200105,93.89,11.47,42.81,272.44,4.43,272.44,5.75,0.17,0.60,...,0.48,9.60,0.54,6.72,0.18,9.06,0.00,Ahmednagar,2001,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9823,202109,96.76,19.47,87.69,269.12,2.00,268.12,3.01,0.92,0.91,...,0.05,8.53,0.08,5.88,11.43,8.45,342.76,Yavatmal,2021,September
9824,202110,97.29,15.75,79.44,37.75,1.36,37.50,2.16,0.90,0.91,...,0.07,6.80,0.07,4.71,3.77,6.73,116.95,Yavatmal,2021,October
9825,202111,97.54,13.61,81.56,69.50,1.66,69.00,2.65,0.80,0.79,...,0.20,5.39,0.27,3.68,0.17,5.12,5.20,Yavatmal,2021,November
9826,202112,97.88,9.70,70.38,70.56,1.45,69.88,2.30,0.70,0.70,...,0.10,5.66,0.12,4.01,0.35,5.54,10.77,Yavatmal,2021,December


import numpy as np
from google.colab import autoviz
df_9207512525578104494 = autoviz.get_df('df_9207512525578104494')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_9207512525578104494, *['PS'], **{})
chart

import numpy as np
from google.colab import autoviz
df_7218176079931694428 = autoviz.get_df('df_7218176079931694428')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_7218176079931694428, *['QV2M'], **{})
chart

import numpy as np
from google.colab import autoviz
df_388332907899550051 = autoviz.get_df('df_388332907899550051')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_388332907899550051, *['RH2M'], **{})
chart

import numpy as np
from google.colab import autoviz
df_424110596106598160 = autoviz.get_df('df_424110596106598160')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_424110596106598160, *['WD2M'], **{})
chart

...

import numpy as np
from google.colab import autoviz
df_8044894011214143515 = autoviz.get_df('df_8044894011214143515')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_8044894011214143515, *[[['PS', 'QV2M'], ['QV2M', 'RH2M'], ['RH2M', 'WD2M'], ['WD2M', 'WS2M']]], **{})
chart

import numpy as np
from google.colab import autoviz
df_3522882089644033214 = autoviz.get_df('df_3522882089644033214')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_3522882089644033214, *['year', 'PS', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_1426270224457805636 = autoviz.get_df('df_1426270224457805636')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_1426270224457805636, *['year', 'QV2M', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_5820086625737963610 = autoviz.get_df('df_5820086625737963610')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_5820086625737963610, *['year', 'RH2M', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_915124969914488227 = autoviz.get_df('df_915124969914488227')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_915124969914488227, *['year', 'WD2M', None], **{})
chart

In [ ]:
state_climate_data.to_csv('/content/drive/MyDrive/MCM Practicum/Implementation/datasets/district_climate_data.csv',
                        header=True,
                        index=False)